## Creacion de una base de datos relacional en PostgreSQL

In [55]:
import sqlite3
import pandas as pd

# 1️⃣ Abrir conexión
connection = sqlite3.connect("bdempresa.db")  # Asegúrate de que esta ruta sea correcta
cursor = connection.cursor()

# 2️⃣ Crear tabla departamentos si no existe
cursor.execute("""
CREATE TABLE IF NOT EXISTS departamentos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL
)
""")

# 3️⃣ Insertar datos de departamentos (evita duplicados usando IGNORE si ya existen)
departamentos = [
    ('Ventas',),
    ('Marketing',),
    ('TI',),
    ('RRHH',)
]

cursor.executemany("""
INSERT OR IGNORE INTO departamentos(nombre) VALUES(?)
""", departamentos)

# 4️⃣ Crear tabla empleados si no existe
cursor.execute("""
CREATE TABLE IF NOT EXISTS empleados (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    edad INTEGER,
    departamento_id INTEGER,
    FOREIGN KEY(departamento_id) REFERENCES departamentos(id)
)
""")

# 5️⃣ Insertar datos de empleados
empleados = [
    ('Juan',28,1),
    ('Ana',22,2),
    ('Juis',32,3),
    ('Maria',29,1),
    ('Carlos',35,4)
]

cursor.executemany("""
INSERT OR IGNORE INTO empleados(nombre,edad,departamento_id) VALUES(?,?,?)
""", empleados)

connection.commit()

# 6️⃣ Verificar que las tablas existan
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print("Tablas existentes:", cursor.fetchall())

# 7️⃣ Hacer un join para ver empleados con su departamento
query = """
SELECT e.id, e.nombre AS empleado, e.edad, d.nombre AS departamento
FROM empleados e
JOIN departamentos d ON e.departamento_id = d.id
"""

df = pd.read_sql_query(query, connection)
display(df)

# 8️⃣ NOTA: cuando termines de trabajar, cerrar la conexión
# connection.close()


Tablas existentes: [('departamentos',), ('sqlite_sequence',), ('empleados',)]


,id,empleado,edad,departamento
0,1,Juan,28,Ventas
1,2,Ana,22,Marketing
2,3,Juis,32,TI
3,4,Maria,29,Ventas
4,5,Carlos,35,RRHH


In [56]:
query='SELECT * FROM empleados'
df=pd.read_sql_query(query, connection)
df

,id,nombre,edad,departamento_id
0,1,Juan,28,1
1,2,Ana,22,2
2,3,Juis,32,3
3,4,Maria,29,1
4,5,Carlos,35,4


In [57]:
query2='SELECT * FROM departamentos'
df2=pd.read_sql_query(query, connection)
df

,id,nombre,edad,departamento_id
0,1,Juan,28,1
1,2,Ana,22,2
2,3,Juis,32,3
3,4,Maria,29,1
4,5,Carlos,35,4


In [58]:
pd.read_sql_query("SELECT COUNT(*) AS total_empleados FROM empleados", connection)
pd.read_sql_query("SELECT COUNT(*) AS total_departamentos FROM departamentos", connection)


,total_departamentos
0,4


In [59]:
query_join = """
SELECT e.nombre AS empleado, e.edad, d.nombre AS departamento
FROM empleados e
JOIN departamentos d ON e.departamento_id = d.id
"""
df_join = pd.read_sql_query(query_join, connection)
df_join


,empleado,edad,departamento
0,Juan,28,Ventas
1,Ana,22,Marketing
2,Juis,32,TI
3,Maria,29,Ventas
4,Carlos,35,RRHH


In [60]:
connection.close()